In [2]:


import cv2
import numpy as np
import mediapipe as mp

In [3]:
mp_drawing = mp.solutions.drawing_utils    
mp_pose = mp.solutions.pose                 

In [7]:
print("Total number of landmarks = {}".format(len(results.pose_landmarks.landmark)))
results.pose_landmarks.landmark[0:2] 

Total number of landmarks = 33


[x: 0.4917893409729004
 y: 0.5736778974533081
 z: -0.6234160661697388
 visibility: 0.9998226165771484,
 x: 0.5185073018074036
 y: 0.5336750745773315
 z: -0.5804299712181091
 visibility: 0.9996976852416992]

In [9]:


count = 0
for i in mp_pose.PoseLandmark:
    print(count, i)
    count += 1

0 PoseLandmark.NOSE
1 PoseLandmark.LEFT_EYE_INNER
2 PoseLandmark.LEFT_EYE
3 PoseLandmark.LEFT_EYE_OUTER
4 PoseLandmark.RIGHT_EYE_INNER
5 PoseLandmark.RIGHT_EYE
6 PoseLandmark.RIGHT_EYE_OUTER
7 PoseLandmark.LEFT_EAR
8 PoseLandmark.RIGHT_EAR
9 PoseLandmark.MOUTH_LEFT
10 PoseLandmark.MOUTH_RIGHT
11 PoseLandmark.LEFT_SHOULDER
12 PoseLandmark.RIGHT_SHOULDER
13 PoseLandmark.LEFT_ELBOW
14 PoseLandmark.RIGHT_ELBOW
15 PoseLandmark.LEFT_WRIST
16 PoseLandmark.RIGHT_WRIST
17 PoseLandmark.LEFT_PINKY
18 PoseLandmark.RIGHT_PINKY
19 PoseLandmark.LEFT_INDEX
20 PoseLandmark.RIGHT_INDEX
21 PoseLandmark.LEFT_THUMB
22 PoseLandmark.RIGHT_THUMB
23 PoseLandmark.LEFT_HIP
24 PoseLandmark.RIGHT_HIP
25 PoseLandmark.LEFT_KNEE
26 PoseLandmark.RIGHT_KNEE
27 PoseLandmark.LEFT_ANKLE
28 PoseLandmark.RIGHT_ANKLE
29 PoseLandmark.LEFT_HEEL
30 PoseLandmark.RIGHT_HEEL
31 PoseLandmark.LEFT_FOOT_INDEX
32 PoseLandmark.RIGHT_FOOT_INDEX


In [5]:
def calc_angle(a,b,c): # 3D points
    
    a = np.array([a.x, a.y]) 
    b = np.array([b.x, b.y])
    c = np.array([c.x, c.y])

    ab = np.subtract(a, b)
    bc = np.subtract(b, c)
    
    theta = np.arccos(np.dot(ab, bc) / np.multiply(np.linalg.norm(ab), np.linalg.norm(bc)))     # A.B = |A||B|cos(x) where x is the angle b/w A and B
    theta = 180 - 180 * theta / 3.14    
    return np.round(theta, 2)

In [12]:
flag = None    
count = 0      


cap = cv2.VideoCapture(0)
pose = mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.5)
while cap.isOpened():
    _, frame = cap.read()

    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)     
    image.flags.writeable = False
    
    results = pose.process(image)                       

    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)     

    try:
        landmarks = results.pose_landmarks.landmark
        shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER]
        elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW]
        wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST]


        angle = calc_angle(shoulder, elbow, wrist)   


        cv2.putText(image,\
                str(angle), \
                    tuple(np.multiply([elbow.x, elbow.y], [640,480]).astype(int)),\
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255),2,cv2.LINE_AA)
    
        if angle > 160:
            flag = 'down'
        if angle < 40 and flag=='down':
            count += 1
            flag = 'up'
        

    except:
        pass


    cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
    cv2.putText(image, str(count), (10,60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)


    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)


    cv2.imshow('MediaPipe feed', image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()